In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from unet import UNet
from keras.datasets.mnist import load_data


e:\ml_stuff\gitcodes\Papers-in-100-Lines-of-Code\Denoising_Diffusion_Probabilistic_Models\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# I would first need access to the samples

(trainX, trainy), (testX, testy) = load_data()
trainX = np.float32(trainX) / 255.
testX = np.float32(testX) / 255.


In [3]:
# I would then need a neural network
mynn = UNet()

In [4]:
# I would need to define the terminal variance and the step size
# 0.1 for sigma_t, not really sure how to choose this
# 0.01 for step size, not really sure how to choose this
delta_t = 0.01
sigma_q = 0.1

In [5]:
# x_0 would be a sample from the data we have
index_0  = torch.randint(low=0,high=60_000,size=(1,))
x_0 = trainX[index_0]


In [6]:
# The uniform sampled time stamp
t = torch.rand(1)

In [7]:
# Create x_t = x_0 + N(0, t * sigma_q)
mean_0 = torch.zeros(28)
cov_x_t = sigma_q* t * torch.eye(28)
gaussian_noise = torch.distributions.MultivariateNormal(mean_0, cov_x_t)
x_t  = gaussian_noise.sample((1,)) + torch.tensor(x_0)


In [8]:
# Create x_{t+delta_t} = x_t + N(0, sigma_q * delta_t)
cov_x_t_delta_t = sigma_q * delta_t * torch.eye(28)
gaussian_noise_delta_t = torch.distributions.MultivariateNormal(mean_0, cov_x_t_delta_t)
x_t_delta_t  = gaussian_noise_delta_t.sample((1,)) + x_t

In [ ]:
# I need to define a loss on this single sample now,
# the loss is ||f_theta(x_{t+delta_t}, t+delta_t) - x_t||^2_2'
# I really need to understand wtf is this f_theta shit now,
# and what it do with this time stuff like t.
# f_theta should really be a neural network.
# so down below, I will start to build my own neuron network
# first of all, we build a resnet block

# The below ResNetBlock will contain the following structure:
# 1. a 3 by 3 conv layer, with padding 1 stride 1, this will give an output of the same dimensionality
# 2. a Batch norm layer
# 3. a ReLU layer
# 4. again a 3 by 3 conv layer with padding 1 stride 1
# 5. a batch norm layer
# 6. an element-wise addition of the thusfar output with the input
# 7. a ReLu layer
# Questions: 
# 1. what is a batch norm?
# 2. how to incorprate the t from the diffusion?
# 3. what is a transposed convolutional layer? (just like a convolutional layer, but this upsamples)

# first is a problem with normalization, if we dont normalize our data, we might have exploding gradient
# second is non-normalized data can significantly decrease our training speed.

# So a batch normalization layer you can choose to apply wherever you want in the network
# The first thing it does is to normalize the output of the current layer

class ResNetBlock(nn.Module):
    def __init__(self, num_input, num_output):
        super().__init__()
        self.convo = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3, stride = 1, padding= 1)
        self.batchnorm = nn.BatchNorm2d(1)
        self.activation = nn.Softmax()

    def forward(self, inputs):
        conv_output = self.convo(inputs)
        batchnorm = self.batchnorm(conv_output)
        output = self.activation(batchnorm)
        return output

myres = ResNetBlock(784, 784)

tensored_x0 =x_0 = torch.tensor(x_0)
x_0 = x_0.unsqueeze(0).unsqueeze(0)
#myres.forward(x_0)

In [11]:
netres = myres.forward(x_0)
netres

C:\Users\dingf\AppData\Local\Temp\ipykernel_30188\883875903.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.activation(batchnorm)


tensor([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 